In [1]:
#this will combine the existing Cost, Salary, and Wage DFs from the MIT data 
#with the county population and then summarize this by state into one state data DF

In [2]:
import pandas as pd

In [3]:
CostDF = pd.read_csv('CostDF.csv')
SalaryDF = pd.read_csv('SalaryDF.csv')
WageDF = pd.read_csv('WageDF.csv')

for d in [CostDF, SalaryDF, WageDF]:
    d.drop(['Unnamed: 0'], axis=1,inplace=True)

In [4]:
#CountyPopUSCensus
#Cpuc
CPUC = pd.read_excel('co-est2022-pop.xlsx', header = [3],
                     skiprows = [4,3149,3150,3151,3152,3153])

In [5]:
issuelist = ['Aleutians West',
 'Bethel',
 'Carson City',
 'Dillingham',
 'East Baton Rouge',
 'Fairbanks North Star',
 'FairfieldConnecticut',
 'Fond du Lac',
 'HartfordConnecticut',
 'Hoonah-Angoon',
 'Isle of Wight',
 'King and Queen',
 'Kusilvak',
 'Lac qui Parle',
 'Lake and Peninsula',
 'Lake of the Woods',
 'Lewis and Clark',
 'LitchfieldConnecticut',
 'MiddlesexConnecticut',
 'New HavenConnecticut',
 'New LondonConnecticut',
 'Nome',
 'Prince of Wales-Hyder',
 'San Luis Obispo',
 'Southeast Fairbanks',
 'TollandConnecticut',
 'Valdez-Cordova',
 'West Baton Rouge',
 'WindhamConnecticut',
 'Yukon-Koyukuk',
            'St. John the Baptist']


def GetCountyName(x):
    n = 0
    CountyList = x.split()
    #print(CountyList[2])
    if ',' in CountyList[2]:
        CountyName = str(CountyList[0][1:]) + ' ' + str(CountyList[1])
        #print(CountyName)
        return CountyName
    #elif 'the' in CountyList[2]:
    #    return 'St. John the Baptist'
    elif ',' not in CountyList[1]:
        CountyList = x.split(',')
        for i in issuelist:
            if i in CountyList[0]:
                return i
    else:
        return CountyList[0][1:]

In [6]:
StateNames = ['Alabama', 'Alaska', 'Arizona', 'Arkansas', 'California', 'Colorado', 
              'Connecticut', 'Delaware', 'Florida', 'Georgia', 'Hawaii', 'Idaho', 
              'Illinois', 'Indiana', 'Iowa', 'Kansas', 'Kentucky', 'Louisiana',
              'Maine', 'Maryland', 'Massachusetts','Michigan', 'Minnesota',
              'Mississippi', 'Missouri', 'Montana', 'Nebraska', 'Nevada', 
              'New Hampshire', 'New Jersey', 'New Mexico', 'New York', 'North Carolina',
              'North Dakota', 'Ohio', 'Oklahoma', 'Oregon', 'Pennsylvania', 
              'Rhode Island', 'South Carolina', 'South Dakota', 'Tennessee', 'Texas', 
              'Utah', 'Vermont', 'Virginia', 'Washington', 'West Virginia', 
              'Wisconsin', 'Wyoming']

In [7]:
def GetStateName(x):
    Names = x.split(',')
    for y in StateNames:
        if 'West Virginia' in Names[-1]:
            return 'West Virginia'
        elif y in Names[-1]:
            return y

In [8]:
CPUC['County'] = [GetCountyName(x) for x in CPUC['Unnamed: 0']]
CPUC['State'] = [GetStateName(x) for x in CPUC['Unnamed: 0']]
CPUC['CountyState'] = CPUC['County'] + CPUC['State']

In [9]:
CPUCdict = CPUC.set_index('CountyState').T.to_dict('list')
#CPUC.where(CPUC['County'] == 'Litchfield').dropna(subset='County') 

C:\Users\chrss\AppData\Local\Temp\ipykernel_24456\3714825379.py:1: UserWarning: DataFrame columns are not unique, some columns will be omitted.
  CPUCdict = CPUC.set_index('CountyState').T.to_dict('list')


In [10]:
fixlist = []
def Get2022Pop(x):
    try:
        b = CPUCdict[str(x)]
    except:
        try:
            a = x.split('ginia')
            newx = a[1][:-3]
            newx = newx + 'West Virginia'
            b = CPUCdict[str(newx)]
        except:
            if 'Connecticut' in x:
                return 453275
            else:
                fixlist.append(x)
                b = [0,1,2,3,0]
    return b[4]

In [11]:
#fixing West Virginia:
#for d in [CostDF, SalaryDF, WageDF]:
#    d['State'] = ['West Virginia' for x in d['County'] if x[:4] is 'ginia']
#    d['County'] = [x.split('ginia')[1][:-3] for x in d['County'] if x[:4] is 'ginia']
#CostDF

In [12]:
for d in [CostDF, SalaryDF, WageDF]:
    d['CountyState'] = d['County'] + d['State']
    d['2022 Population Est'] = [Get2022Pop(str(x)) for x in d['CountyState']]
    d.drop(['CountyState'],axis=1,inplace=True)


In [13]:
fixlist = set(fixlist)
fixlist

{'BethelAlaska',
 'Carson CityNevada',
 'DillinghamAlaska',
 'Hoonah-AngoonAlaska',
 'KusilvakAlaska',
 'NomeAlaska',
 'Valdez-CordovaAlaska',
 'Yukon-KoyukukAlaska'}

In [14]:
SalaryDF

,Occupational Area,Typical Annual Salary,State,County,2022 Population Est
0,Management,"$101,778",Alabama,Autauga,59759
1,Business &amp; Financial Operations,"$71,198",Alabama,Autauga,59759
2,Computer &amp; Mathematical,"$86,938",Alabama,Autauga,59759
3,Architecture &amp; Engineering,"$90,447",Alabama,Autauga,59759
4,"Life, Physical, &amp; Social Science","$64,992",Alabama,Autauga,59759
...,...,...,...,...,...
69735,"Farming, Fishing, &amp; Forestry","$33,534",Wyoming,Weston,6860
69736,Construction &amp; Extraction,"$55,501",Wyoming,Weston,6860
69737,"Installation, Maintenance, &amp; Repair","$59,503",Wyoming,Weston,6860
69738,Production,"$55,854",Wyoming,Weston,6860


In [15]:
WageDF.to_csv('WageDFwithPop.csv')
CostDF.to_csv('CostDFwithPop.csv')
SalaryDF.to_csv('SalaryDFwithPop.csv')